In [6]:
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.metrics import accuracy_score, precision_score,recall_score,f1_score

In [7]:
df = pd.read_csv('outputs/dataset_players_statistics.csv')
df

,game,blueTopGP,blueTopWR,blueTopKDA,blueJungleGP,blueJungleWR,blueJungleKDA,blueMidGP,blueMidWR,blueMidKDA,...,redMidGP,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA,result
0,ESPORTSTMNT03/1632489,0,0.00,0.00,14,0.57,7.45,0,0.00,0.00,...,3,0.33,10.67,1,0.00,13.00,4,0.25,4.45,0
1,ESPORTSTMNT03/1632500,2,0.50,2.50,4,0.75,6.40,0,0.00,0.00,...,12,0.75,5.21,3,0.67,11.67,2,0.50,9.67,1
2,ESPORTSTMNT03/1632502,3,0.33,2.33,5,0.60,4.50,1,0.00,8.00,...,0,0.00,0.00,4,0.50,9.80,3,0.33,8.40,1
3,6909-9183,0,0.00,0.00,3,0.33,5.00,4,0.50,12.25,...,0,0.00,0.00,0,0.00,0.00,0,0.00,0.00,1
4,6909-9184,0,0.00,0.00,2,0.50,3.00,6,0.33,4.17,...,1,0.00,7.00,1,0.00,8.00,0,0.00,0.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7724,ESPORTSTMNT01/2138172,0,0.00,0.00,1,0.00,14.00,0,0.00,0.00,...,0,0.00,0.00,0,0.00,0.00,0,0.00,0.00,1
7725,ESPORTSTMNT01/2148437,4,0.75,4.29,5,0.60,3.92,4,0.75,5.78,...,1,0.00,12.00,4,0.25,10.80,1,0.00,1.71,1
7726,ESPORTSTMNT01/2138187,0,0.00,0.00,0,0.00,0.00,1,0.00,0.50,...,1,1.00,8.50,1,1.00,21.00,1,1.00,7.67,1
7727,ESPORTSTMNT01/2148443,6,0.67,3.92,5,0.60,2.83,3,0.67,2.33,...,0,0.00,0.00,2,0.50,6.50,6,0.50,2.04,0


In [8]:
y = df['result'].copy()
X = df.drop(['result', 'game'],axis=1)
X

,blueTopGP,blueTopWR,blueTopKDA,blueJungleGP,blueJungleWR,blueJungleKDA,blueMidGP,blueMidWR,blueMidKDA,blueADCGP,...,redJungleKDA,redMidGP,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA
0,0,0.00,0.00,14,0.57,7.45,0,0.00,0.00,2,...,3.11,3,0.33,10.67,1,0.00,13.00,4,0.25,4.45
1,2,0.50,2.50,4,0.75,6.40,0,0.00,0.00,4,...,3.43,12,0.75,5.21,3,0.67,11.67,2,0.50,9.67
2,3,0.33,2.33,5,0.60,4.50,1,0.00,8.00,0,...,6.67,0,0.00,0.00,4,0.50,9.80,3,0.33,8.40
3,0,0.00,0.00,3,0.33,5.00,4,0.50,12.25,0,...,1.50,0,0.00,0.00,0,0.00,0.00,0,0.00,0.00
4,0,0.00,0.00,2,0.50,3.00,6,0.33,4.17,1,...,0.00,1,0.00,7.00,1,0.00,8.00,0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7724,0,0.00,0.00,1,0.00,14.00,0,0.00,0.00,0,...,0.00,0,0.00,0.00,0,0.00,0.00,0,0.00,0.00
7725,4,0.75,4.29,5,0.60,3.92,4,0.75,5.78,3,...,7.00,1,0.00,12.00,4,0.25,10.80,1,0.00,1.71
7726,0,0.00,0.00,0,0.00,0.00,1,0.00,0.50,1,...,0.00,1,1.00,8.50,1,1.00,21.00,1,1.00,7.67
7727,6,0.67,3.92,5,0.60,2.83,3,0.67,2.33,8,...,0.00,0,0.00,0.00,2,0.50,6.50,6,0.50,2.04


In [9]:
def preprocess_input(X,y):
    X = X.copy()
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=42, stratify=y)
    scaler = StandardScaler()   
    print(X_train) 
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train,X_test,y_train,y_test
X_train,X_test,y_train,y_test = preprocess_input(X,y)

      blueTopGP  blueTopWR  blueTopKDA  blueJungleGP  blueJungleWR  \
1861          1       0.00        2.00             1           0.0   
4117          1       1.00       14.00             0           0.0   
1901          3       0.33       19.00             2           0.5   
2064          1       1.00       15.00             0           0.0   
5304          8       0.38        4.48             2           1.0   
...         ...        ...         ...           ...           ...   
229           1       1.00        1.00             0           0.0   
5092          2       0.50        2.57             1           1.0   
2905          1       1.00       12.00             1           0.0   
1562          0       0.00        0.00             2           0.0   
1341          1       1.00        8.00             0           0.0   

      blueJungleKDA  blueMidGP  blueMidWR  blueMidKDA  blueADCGP  ...  \
1861           0.00          2        0.5        3.25          1  ...   
4117         

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier

models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Support Vector Machine (Linear Kernel)': LinearSVC(),
    'Support Vector Machine (RBF Kernel)': SVC(),
    'Decission Tree': DecisionTreeClassifier(),
    'Adaboost': AdaBoostClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting Classifier': GradientBoostingClassifier()
}

GBCmodel = GradientBoostingClassifier()
GBCmodel.fit(X_train,y_train)


for name, model in models.items():
  model.fit(X_train,y_train)
  print(name + ' trained')

Logistic Regression trained
C:\Users\linco\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
Support Vector Machine (Linear Kernel) trained
Support Vector Machine (RBF Kernel) trained
Decission Tree trained
Adaboost trained
Random Forest trained
Gradient Boosting Classifier trained


In [11]:
scores_list = []

for name,model in models.items():    
    scores_list.append({
    'Model': name,
    'Accuracy': accuracy_score(y_test,model.predict(X_test)),
    'Precision':  precision_score(y_test,model.predict(X_test)),
    'Recall': recall_score(y_test,model.predict(X_test)),
    'F1-Score': f1_score(y_test,model.predict(X_test))
    })
scores = pd.DataFrame(scores_list)

In [110]:
scores

,Model,Accuracy,Precision,Recall,F1-Score
0,Logistic Regression,0.926437,0.921182,0.921182,0.921182
1,Support Vector Machine (Linear Kernel),0.927586,0.921376,0.923645,0.922509
2,Support Vector Machine (RBF Kernel),0.924138,0.912621,0.926108,0.919315
3,Decission Tree,0.877011,0.863747,0.874384,0.869033
4,Adaboost,0.931034,0.932500,0.918719,0.925558
5,Random Forest,0.934483,0.950904,0.906404,0.928121
6,Gradient Boosting Classifier,0.945977,0.943210,0.940887,0.942047
